<a href="https://colab.research.google.com/github/IBM-EPBL/IBM-Project-22038-1659801788/blob/main/3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import cv2
import shutil
import random as rn
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir ="/content/drive/MyDrive/Colab Notebooks/flowers"

In [ ]:
print(os.listdir("/content/drive/MyDrive/Colab Notebooks/flowers"))

['flowers']


In [ ]:
batch_size = 32
img_height = 180
img_width = 180
num_classes = 5

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4331 files belonging to 1 classes.
Using 3465 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4331 files belonging to 1 classes.
Using 866 files for validation.


In [ ]:
class_names = train_ds.class_names
class_names

['flowers']

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.008626422 0.9963726


* ### ***Image Augmentation***





In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

*   ### ***Model Creation / Adding Layers***

In [ ]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.3),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(num_classes)
])

* ### ***Compling the Model***

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

* ### ***Fitting the Model***

In [ ]:
epochs=25
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/25
109/109 [==============================] - 150s 1s/step - loss: 0.0191 - accuracy: 0.9925 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/25
109/109 [==============================] - 5s 44ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/25
109/109 [==============================] - 5s 43ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/25
109/109 [==============================] - 5s 43ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/25
109/109 [==============================] - 5s 43ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/25
109/109 [==============================] - 5s 43ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/25
109/109 [==============================] - 5s 44ms/step - loss: 0.0000e+00 - accuracy: 

* ### ***Testing on unseen image Data***

In [ ]:
from matplotlib import image as im
from matplotlib import pyplot
from keras.preprocessing import image
from PIL import Image
data = im.imread('/content/drive/MyDrive/Colab Notebooks/th.jpg')
img=image.load_img('/content/drive/MyDrive/Colab Notebooks/th.jpg', target_size=(180, 180))

test_img=np.expand_dims(img , axis=0)
  
result = model.predict(test_img)
pred = np.argmax(result) 
print(result)
print(pred)
print(class_names)



FileNotFoundError: ignored

In [ ]:
image = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/Colab Notebooks/IBM/th (1).jpg', target_size=(180,180))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  
result = model.predict(input_arr)
pred = np.argmax(result) 
print(class_names)
print(pred)
print(class_names[pred])

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
3
sunflower


* ### ***Saving Model***

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/IBM/flower_model.h5")

In [ ]:
from tensorflow.keras.models import load_model

savedModel=load_model("/content/drive/MyDrive/Colab Notebooks/IBM/flower_model.h5")
savedModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 90, 90, 16)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  